# Set parameters & Load data

In [ ]:
import pandas as pd
import numpy as np
np.random.seed(666)
import matplotlib.pyplot as plt
from tqdm import tqdm
tqdm.pandas()

from sksurv.nonparametric import kaplan_meier_estimator

# from lifelines import KaplanMeierFitter
from lifelines import CoxPHFitter

%config InlineBackend.figure_format ='retina'
pd.set_option('display.max_columns', None)

In [ ]:
%load_ext autoreload
%autoreload 2
from constants import *

In [ ]:
from helper import *
from basic_feature_extraction import *
from data_extraction import *
from bc_comparator import *
from ps_estimator import *
from eGFR_mean_curve import *
from KM_estimator import *
from cox_regression import *

In [ ]:
#load data
eGFR_df = pd.read_pickle(PRO_PATH+BIO_COL+'_df.pklz', compression = 'gzip')
first_last_date_df = pd.read_pickle(PRO_PATH+'first_last_date_df.pkl')
DHL_date_df = pd.read_pickle(RAW_PATH+'patType_diagnosticDate.pklz', compression = 'gzip')
medHTN_df = pd.read_pickle(PRO_PATH+'medHTN_class_df_GQ_classification.pklz', compression = 'gzip')

In [ ]:
medDM_df = pd.read_pickle(PRO_PATH+'medDM_df.pklz', compression = 'gzip')
medHLD_df = pd.read_pickle(PRO_PATH+'medHLD_df.pklz', compression = 'gzip')

In [ ]:
#Total paitent numbers
patient_df = pd.read_pickle(RAW_PATH + 'vector_PatientBased.pklz', compression = 'gzip')
patient_df.shape

#  Extract data

In [ ]:
def data_extraction(medHTN_df, eGFR_df, DHL_date_df):
    
    if 'drug_intensity' in GRP_LIST:
        print('Mapping drug dosage to drug intensity...')
        treatment_hist_df = extract_HTN_treatment_history(medHTN_df)
    
        print('Extracting initial HTN drug prescription...')
        drug_class_df = extract_init_drug_class_with_intensity_hist(treatment_hist_df, DRUG_LIST)
        
    else:
        drug_class_df = extract_init_drug_class(medHTN_df)

    print('Extracting eGFR change date...')
    print(f'eGFR_OPTION: {eGFR_OPTION}')
    stage_changing_df = extract_eGFR_stage_changing(eGFR_df, drug_class_df, option = eGFR_OPTION)

    print('Extracting HTN diagnostic date...')
    HTN_date_df = get_HTN_diagnostic_date(DHL_date_df)
    
    print(f'Associating baseline and followup values of {BIO_COL}')
    bio_df = pd.read_pickle(PRO_PATH+ BIO_COL +'_df.pklz', compression = 'gzip')
    followups_ctrl_df = get_followups_with_control(drug_class_df, bio_df)
    

    return drug_class_df, stage_changing_df, HTN_date_df, followups_ctrl_df
# %%time
# drug_class_df, stage_changing_df, HTN_date_df = data_extraction(medHTN_df, eGFR_df, DHL_date_df)

# Cohort derivation

In [ ]:
def get_in_cohort_remark(row, exclude_pids):

    # age
    if row['Age'] < 21:
        return '2 - Age'
    
    # not diagnose with HTN before index date
    if (row['HTN_date'] > row['D_start_date']):
        return '3 - HTN drugs before diagnosis'
    
    # not initial therapy -> False
    #condition: row['D_start_date'] < '2010-06-30', less than 365 days from the study period
    if row['D_start_date'] < pd.to_datetime('2011-01-01') :
        return '4 - the 1st therapy was not drug initailization'
    
    if row[PID] in exclude_pids:
        return '5 - taking excluded drugs'

    # combined therapy -> False
    if row['drug_class'] == 'OutOfSelectedHTNClasses':
        return '6 - OutOfSelectedHTNClasses'
    
    if row['drug_name'] == 'Propranolol':
        return '7 - Propranolol_HTNexcluded'

    if row['drug_num'] > 1:
        return '8 - HTN drugs more than 1'

    #invalid eGFR record -> False
    if row['B_remark'] in ['single record', 'no baseline eGFR', 'no follow-up eGFR']:
        return '9 - no eligible baseline or follow-up eGFR'
    
    if (row['D_start_date'] - row['B_init_date']).days//365 > 0:
        return '10 - Baseline eGFR out of window'
    
    #followups with ctrl
    if len(row['follow_ups_with_control']) == 0:
        return '11 - No eligible control eGFR'


#     # if diagnosed with kidney complication -> False
#     if (row['Kidney_diag'] == True):
#         return 'Kidney_CC'
    
    return 'IN COHORT'
    
def cohort_derivation(drug_class_df,
                      stage_changing_df,
                      HTN_date_df,
                      first_last_date_df,
                      followups_ctrl_df,
                      exclude_pids):
    
    #join related df on PID with inner join
    df_list = [drug_class_df, stage_changing_df, HTN_date_df, first_last_date_df, followups_ctrl_df]
    joined_df = join_df_list_on_same_key(df_list,[PID], how='inner')
    
    date_attr = 'D_start_date'
    pid_date_df = joined_df[[PID, date_attr]]
    
    #add age at `D_start_date`
    age_df = add_age_at_date(pid_date_df, date_attr = date_attr)
    
    #add kidney complication at `D_start_date`
    kidney_df = adding_CC_diagnostic_variables(pid_date_df,
                                               date_attr = date_attr, 
                                               CC_list = ['Kidney'])
    
    joined_df = join_df_list_with_change_date(pid_date_df,
                                              [joined_df, age_df, kidney_df],
                                              date_attr = date_attr)
    
    #assign cohort mark
    joined_df['in_cohort'] = joined_df.progress_apply(lambda row: get_in_cohort_remark(row, exclude_pids), axis=1)
  
    #cohort analysis
    cohort_summary = joined_df['in_cohort'].value_counts().sort_index()
    print('In cohort summary: \n', cohort_summary)
    
    cohort_df = joined_df.loc[joined_df['in_cohort'] == 'IN COHORT', :].reset_index(drop = True)
    
    #POST-PROCESSING
    #combine the multi-col-identifier into one col
    cohort_df = generate_grp_tuple_col(cohort_df, GRP_LIST)
    cohort_df.loc[:, GRP_COL] = cohort_df[GRP_COL].apply(lambda v: CLASS_MAPPING_DICT[v])
    #generating survival labels
    event_dur_df = cohort_df.apply(lambda row: get_event_dur(row), axis = 1)
    cohort_df = cohort_df.merge(event_dur_df, how = 'inner', left_on = PID, right_on = PID)
    
    print('In cohort summary: #total_patients.')
    print(len(cohort_df), '\n')
    
    #summary of eGFR stage change of patients in cohort
    summary_eGFR = cohort_df['B_remark'].value_counts().sort_values()
    print('In cohort summary: #patient for eGFR change event.')
    print(summary_eGFR, '\n')
    
    #summary of drug prescription of patients in cohort
    summary_drug_intensity = cohort_df[GRP_COL].value_counts().sort_values(ascending = False)
    print('In cohort summary: #patient for each group.')
    print(summary_drug_intensity, '\n')
    
    return cohort_df

# %%time
# cohort_df = cohort_derivation(drug_class_df, stage_changing_df, HTN_date_df, first_last_date_df)

# #time from baseline eGFR measurement to drug initiation - by year
# (cohort_df['D_start_date'] - cohort_df['B_init_date']).apply(lambda v:v.days//365).value_counts()

# #duration of drug prescription
# (cohort_df['D_end_date'] - cohort_df['D_start_date']).apply(lambda v: v.days).hist(bins = 20)

# #duration from drug end to biomarker change
# #to see how much of biomarker change after the stop of drug
# (cohort_df['B_change_date'] - cohort_df['D_end_date']).apply(lambda v: v.days/365).hist(bins = 20)

# #to see how much of biomarker change after the stop of drug
# cohort_df.loc[(cohort_df['B_change_date'] > cohort_df['D_end_date']) & (cohort_df['B_remark'] != 'no change') ,:].head(10)

## Add baseline covaraites

In [ ]:
def associating_baseline_vars_df(cohort_df, eGFR_option):
    
    date_attr = 'D_start_date'
    pid_date_df = cohort_df[[PID, date_attr]].reset_index(drop = True)
    df_list = []
    dummies_prefixes = []
    
    drug_class_df = cohort_df[[PID, date_attr] + [GRP_COL]]
    df_list.append(drug_class_df)
    
    sex_race_df = adding_gender_race_variables(pid_date_df, 
                                          date_attr = date_attr, 
                                          with_dummies = True,
                                          drop_ref = DUMMIES_DROP_FIRST)
    df_list.append(sex_race_df)
    dummies_prefixes += ['gender', 'race']
    
#     #add age as a discrete varaible
#     age_band_df = add_age_band_at_date(pid_date_df, 
#                                         date_attr = date_attr,
#                                         with_dummies = True,
#                                         drop_ref = DUMMIES_DROP_FIRST)
#     df_list.append(age_band_df)
#     dummies_prefixes += ['age']
    
    #add age as a continuous variable
    age_df = add_age_at_date(pid_date_df, 
                            date_attr = date_attr)
    df_list.append(age_df)
    

    DHL_df = adding_DHL_diagnostic_variables(pid_date_df, 
                                             date_attr = date_attr,
                                            DHL_list = ['DM', 'HLD'])
    df_list.append(DHL_df)
    
    CC_df = adding_CC_diagnostic_variables(pid_date_df,
                                           date_attr = date_attr,
                                           CC_list = ['Macrovascular','Eye','Foot','Kidney'])
    df_list.append(CC_df)
    
    lab_list = ['BP_S', 'BP_D', 'HbA1c', 'LDL', 'BMI']
    lab_df = adding_baseline_lab_values_from_lab_df_opt(pid_date_df, 
                                           lab_list, 
                                           date_attr = date_attr, 
                                           wind_size=180,
                                           fillna = True)

    df_list.append(lab_df)
    
    drug_df = adding_baseline_drug_variables_opt(pid_date_df, 
                                       DHL_list = ['DM', 'HLD'],
                                       date_attr = date_attr, 
                                       wind_size = 180)
    df_list.append(drug_df)
    
    if eGFR_option == 'stage':
        eGFR_dummies = pd.get_dummies(cohort_df['B_init_value'].astype(int), prefix='eGFR_stage', drop_first = DUMMIES_DROP_FIRST)\
                            .reset_index(drop = True)
        init_stage_df = pd.concat([pid_date_df, eGFR_dummies], axis = 1)
        df_list.append(init_stage_df)
        dummies_prefixes += ['eGFR_stage']
    else: 
        init_eGFR_df = cohort_df[[PID, date_attr, 'B_init_value']].rename(columns = {'B_init_value': 'eGFR'})
        df_list.append(init_eGFR_df)

    baseline_vars_df = join_df_list_with_change_date(pid_date_df,
                                              df_list,
                                              date_attr = date_attr)
    
    
    return baseline_vars_df, dummies_prefixes


# %%time
# baseline_vars_df, dummies_prefixes = associating_baseline_vars_df(cohort_df, ['drug_class', 'drug_intensity'], eGFR_option = eGFR_OPTION)

# baseline_vars_df.columns

## PACKED 

In [ ]:
def cohort_derivation_and_comparision(drug_class_df, 
                                      stage_changing_df, 
                                      HTN_date_df, 
                                      first_last_date_df, 
                                      followups_ctrl_df, 
                                      exclude_pids):
    
    print('Cohort derivation...')
    cohort_df = cohort_derivation(drug_class_df, 
                                  stage_changing_df, 
                                  HTN_date_df, 
                                  first_last_date_df,
                                  followups_ctrl_df,
                                  exclude_pids)
    
    print('\n Associating baseline variables...')
    baseline_vars_df, dummies_prefixes = associating_baseline_vars_df(cohort_df,
                                                                      eGFR_option = eGFR_OPTION)

    #select patients with condition
    print(f'\n cohort_DM: {cohort_DM}, cohort_CKD: {cohort_CKD}')
    if cohort_DM is None:
        mask_DM = np.array([True] * len(baseline_vars_df))
    else:
        mask_DM = baseline_vars_df['DM_diag'] == cohort_DM
        
    if cohort_CKD is None:
        mask_CKD = np.array([True] * len(baseline_vars_df))
    else:
        mask_CKD = baseline_vars_df['Kidney_diag'] == cohort_CKD

    pids = baseline_vars_df.loc[mask_DM & mask_CKD, PID].unique()
    
    cohort_df_s = cohort_df.loc[cohort_df[PID].isin(pids), :]
    baseline_vars_df_s = baseline_vars_df.loc[baseline_vars_df[PID].isin(pids), :]
    
    print('\n Comparing sub-cohorts...')
    bc_df = get_baseline_characteristics(baseline_vars_df_s)

    cohort_comparison(baseline_vars_df_s)

    return cohort_df_s, baseline_vars_df_s, bc_df, dummies_prefixes

# eGFR curve overtime

# Kaplan-Meier estimation

In [ ]:
def KM_estimator_for_params(cohort_df):
    
    event_params = {}
    for window in ['drug', 'study']:
        for event in ['better', 'worse']:
            event_params['window'] = window
            event_params['eGFR_event'] = event
            KM_estimator(cohort_df, event_params)
    
    return

#  Cox proportionl hazard regression

In [ ]:
def Cox_for_multiple_events(baseline_vars_df, cohort_df):
    
    event_params = {'window': 'drug', 
               'eGFR_event': 'better'}
    
    for events in ['better', 'worse', 'fluctuate']:
        
        print(f'-------------------eGFR_event: {events.upper()}----------------')
        event_params['eGFR_event'] = events
        cox_surv_df = adding_dur_event_variables_with_parameters(baseline_vars_df,
                                                             cohort_df,
                                                             event_params)
    
        cox_weights_dict, res_df = Cox_for_multiple_base_classes(cox_surv_df)
        print(res_df.to_string())

    return

# %%time
# Cox_for_multiple_events(baseline_vars_df, cohort_df)

# Propensity Score for covaraites balancing

## PACKED: PS compuation

In [ ]:
def ps_computation(baseline_vars_df):
    
    print('----------Estimating propensity score...')
    ps_dict = ps_for_all_classes(baseline_vars_df, drop_ref = True)

    print('----------Positivity checking ...')
    ps_positivity_dict = positivity_checking(ps_dict)

    return ps_positivity_dict

# ps_positivity_dict = ps_computation(baseline_vars_df)

## PS stratification

def generate_stratum_label(ps_dict, strata_num):
    
    strata_dict = {}
    for key, df in ps_dict.items():
        if df.empty:
            continue
        ps_series = df.loc[:, 'propensity']
        df.loc[:, 'stratum'] = pd.qcut(ps_series, strata_num, labels = False)
        strata_dict[key] = df

    return strata_dict

def SMD_with_strata(strata_df, base, comparator):
    
    stratums = strata_df['stratum'].unique()
    cols = list(strata_df.columns)
    
    SMD_list = []
    for label in stratums:
        s_df = strata_df.loc[strata_df['stratum'] == label, :]
        X_cols = remove_element_from_list(cols, ['propensity', 'stratum'])
        SMD_df = SMD_between_base_and_comparator(s_df[X_cols], base, comparator)
        SMD_list.append(SMD_df[['SMD']])
        
    full_df = pd.concat(SMD_list, axis = 1)
    res_df = full_df.mean(axis = 1)

    return res_df

def cohort_comparison_after_stratification(strata_dict, details = False):
    
    for key, strata_df in strata_dict.items():
        if strata_df.empty:
            continue
        base, comparator = key
        imbalanced_cnt = 0
        SMD_s = SMD_with_strata(strata_df, base, comparator)
        SMD_s = SMD_s.round(2)
        imbalanced_rows = SMD_s.loc[SMD_s>=0.2]
        imbalanced_cnt += len(imbalanced_rows)
        
        print(f'----------base: {base}, comparator: {comparator}----------')
        if details:
            print(imbalanced_rows)
            print(f'#imbalanced_row: {imbalanced_rows}')
        else:
            pass
        
    print(f'Total imbalanced rows: {imbalanced_cnt}')
        
    return

# ps_positivity_dict = positivity_checking(ps_dict)

# strata_dict = generate_stratum_label(ps_positivity_dict, strata_num = 20)

# cohort_comparison_after_stratification(strata_dict)

## PACKED: Cox with stratification

In [ ]:
def Cox_strata_for_multiple_events(ps_dict, cohort_df, dummies_prefixes):
    
    print('----------Stratification ...')
    strata_dict = generate_stratum_label(ps_dict, strata_num = 20)

    print('----------Cohort comparison after stratification ...')
    cohort_comparison_after_stratification(strata_dict)
    
    print('----------Cox regression ...')
    
    event_params = {'window': 'drug', 
               'eGFR_event': 'better'}
    
    for events in ['better', 'worse']:
        
        print(f'-------------------eGFR_event: {events.upper()}----------------')
        event_params['eGFR_event'] = events
        res_df = cox_with_strata(strata_dict,
                                    cohort_df,
                                    dummies_prefixes,
                                    event_params)

    return

# %%time
# Cox_strata_for_multiple_events(ps_positivity_dict, cohort_df)

## PACKED: Cox with PS-matching

In [ ]:
def Cox_ps_matching_for_multiple_events(ps_dict, cohort_df, dummies_prefixes):
    
    print('-------------------PS matching-------------------')
    matched_dict = one_to_one_ps_matching_for_all_pairs(ps_dict)
    
    event_params = {'window': 'drug', 
               'eGFR_event': 'better'}
    
    for events in ['better', 'worse']:
        
        print(f'-------------------eGFR_event: {events.upper()}----------------')
        event_params['eGFR_event'] = events
        
        cox_with_matching(ps_dict,
                          cohort_df,
                          matched_dict,
                          dummies_prefixes,
                          event_params)

    return

# %%time
# Cox_ps_matching_for_multiple_events(ps_positivity_dict, cohort_df)

# KM with ps-matching

In [ ]:
def ps_matching_for_KM_estimator(baseline_vars_df, comparator):
    
    matched_dict = ps_estimation_and_matching(baseline_vars_df, comparator = comparator)
    bc_comp_dict = profile_comparison_after_matching(baseline_vars_df, matched_dict)
    SMD_visualization(bc_comp_dict)
 
    return matched_dict, bc_comp_dict

def pairwise_KM_estimator(cohort_df, matched_dict):
    
    for key, matched in matched_dict.items():
        base, comparator = key
#         print(key)
        base_pids = matched['base_id'].to_list()
        comp_pids = matched['comp_id'].to_list()
        pid_list = base_pids + comp_pids
#         print(f'sample of base_pids: {np.random.choice(base_pids, size = 5)}')
#         print(f'sample of comp_pids: {np.random.choice(comp_pids, size = 5)}')
        
        mask = cohort_df[PID].isin(pid_list)
        cohort_df_s = cohort_df.loc[mask, :]

        sample_size = len(matched)
        additional_title = f'(sample_size:{sample_size})'
        
        
        event_params = {'window': 'drug', 
                        'eGFR_event': 'worse'}
        KM_estimator(cohort_df_s,
                     event_params,
                     additional_title = additional_title)

    return 

# ALL PACKED

In [ ]:
# %%time
# treatment_hist_df = extract_HTN_treatment_history(medHTN_df)

In [ ]:
# %%time
# inter_v_df = eGFR_mean_curve_entire_population(eGFR_df)

## cohort extraction

In [ ]:
%%time
drug_class_df, eGFR_changing_df, HTN_date_df, followups_ctrl_df = data_extraction(medHTN_df, eGFR_df, DHL_date_df)

In [ ]:
drug_class_df.head()

In [ ]:
eGFR_changing_df.head()

In [ ]:
HTN_date_df.head()

In [ ]:
followups_ctrl_df.head()

In [ ]:
drug_class_df['drug_num'].value_counts().iloc[0:10]

In [ ]:
HTN_date_df.shape

In [ ]:
%%time
excluded_pids = get_pids_taking_certain_drugs(medDM_df, ['Empagliflozin', 'Dapagliflozin'])
cohort_df, baseline_vars_df, bc_df, dummies_prefixes = cohort_derivation_and_comparision(drug_class_df,
                                                                                           eGFR_changing_df,
                                                                                           HTN_date_df,
                                                                                           first_last_date_df,
                                                                                           followups_ctrl_df,
                                                                                           excluded_pids)

In [ ]:
cohort_df.columns

In [ ]:
cohort_df['HTN_date'].isnull().value_counts()

In [ ]:
cohort_df.groupby(GRP_COL).apply(lambda df: print(df.name, len(df), df['event'].sum()))

In [ ]:
cohort_df.head()

In [ ]:
cohort_df.shape

In [ ]:
cohort_df['grp_name'].unique()

In [ ]:
#generate the pivot table
def pivot_table_for_stage_transition(cohort_df):

    with pd.ExcelWriter('CKD_stage_change.xlsx') as writer:
        copy_df = cohort_df.copy()
        for grp in ['ACEIs', 'ARBs', 'OTHERS']:
            s_df = copy_df.loc[copy_df['grp_name'] == grp, ['B_init_value','B_worse_value']]
            s_df['B_worse_value'] = s_df.apply(lambda row: row['B_init_value'] if pd.isnull(row['B_worse_value']) else row['B_worse_value'], axis = 1)  
            pivot_table = pd.pivot_table(s_df, index = 'B_init_value', columns = 'B_worse_value', aggfunc = 'size')
            pivot_table = pivot_table.fillna(0).astype(int)
            pivot_table['ALL'] = pivot_table.sum(axis = 1)
            pivot_table = (pivot_table.div(pivot_table['ALL'], axis = 0) * 100).round(1)
#             print(grp)
#             print(pivot_table)

    #         pivot_table.to_excel(writer, sheet_name=grp, index=True)
    
    return

pivot_table_for_stage_transition(cohort_df)

In [ ]:
#mean follow-up years, and IQR
(cohort_df['D_end_date'] - cohort_df['D_start_date']).apply(lambda v: v.days/365).median()

In [ ]:
cohort_df[GRP_COL].value_counts(normalize = True)

In [ ]:
#eGFR measurement frequency for those in cohort
def compute_eGFR_measurement_freq(eGFR_df,pids):
    
    eGFR_df_s = eGFR_df.loc[eGFR_df[PID].isin(pids), :].reset_index(drop = True)
    interval = eGFR_df_s.groupby(PID).apply(lambda df: df[DATE].diff())
    
    return interval.median()

pids = cohort_df[PID].unique()
compute_eGFR_measurement_freq(eGFR_df,pids)

In [ ]:
# summary = baseline_vars_df.loc[baseline_vars_df[GRP_COL] == 'class_ARB', :]\
#                                 .drop([PID, GRP_COL], axis = 1)\
#                                 .sum()[['eGFR_stage_1','eGFR_stage_2','eGFR_stage_3','eGFR_stage_4','eGFR_stage_5']]
# summary/3171

In [ ]:
bc_df

In [ ]:
#statistics
baseline_vars_df['gender_Male'].mean()

In [ ]:
baseline_vars_df.head()

In [ ]:
# #save to csv
# bc_df.to_csv('bc.csv')

In [ ]:
# baseline_vars_df.groupby(['DM_diag', 'HLD_diag']).size()

In [ ]:
#adding traj for other variables
traj_df = get_followups_with_control_addtional_traj(cohort_df, traj_col = TRAJ_COL)
cohort_df = cohort_df.merge(traj_df, how = 'left', left_on = PID, right_on = PID)

In [ ]:
traj_df.shape

In [ ]:
traj_df.head()

In [ ]:
cohort_df.columns

### distribution of baseline eGFR

In [ ]:
def get_eGFR_stage(v):
    
    if v >= 90:
        return 'G1'
    elif v >= 60:
        return 'G2'
    elif v >=30:
        return 'G3'
    elif v >= 15:
        return 'G4'
    else:
        return 'G5'

def distribution(baseline_vars_df):

    col = 'eGFR'
    without_CKD = False
    
    if without_CKD:
        s_df = baseline_vars_df.loc[baseline_vars_df['Kidney_diag'] == False, :]
    else:
        s_df = baseline_vars_df
    
    stages = s_df[col].apply(lambda v:get_eGFR_stage(v))
    
    print(stages.value_counts())
    
    stages.hist()
    
    return

In [ ]:
# distribution(baseline_vars_df)

## ps-estimation

**Input:**
* `baseline_vars_df`

**Output:**
* `ps_positivity_dict` = (PID, GRP_COL, 'ps', 'ps_w')

In [ ]:
# %%time
# ps_positivity_dict = estimate_propensity_score(baseline_vars_df, 
#                                               comp_list = ['class_ACEI'])

## eGFR mean curve

### mean curve with control point & IPTW

In [ ]:
def eGFR_mean_IPTW(followups_df, baseline_vars_df, followup_yrs):

    followups_df_s, baseline_vars_df_s = select_patient_with_followup_yrs(followups_df, 
                                                                         baseline_vars_df, 
                                                                         followup_yrs)
    
    ps_positivity_dict = estimate_propensity_score(baseline_vars_df_s, 
                                                   comp_list = ['ACEIs', 'ARBs'])
    
    bc_comp_dict = profile_comparison_IPTW(baseline_vars_df_s, ps_positivity_dict)
    SMD_visualization(bc_comp_dict, followup_yrs)
    
    for pair, ps_df in ps_positivity_dict.items():
        pair_stats_dict = get_pair_stats(followups_df_s, ps_df)
        followups_visualization(pair_stats_dict,
                                ctrl_expolation = False,
                                add_patient_num = True,
                                errorbar = True)
        
        print(f'-----------------------{pair}------------------------')
        for key, grp_dict in pair_stats_dict.items():
            print(key)
            print('means: ', np.round(grp_dict['means'],2))
            print('CI_lower: ', np.round(grp_dict['CI_lower'],2))
            print('CI_upper: ', np.round(grp_dict['CI_upper'],2),'\n')

    return bc_comp_dict

In [ ]:
%%time
followups_df = cohort_df[[PID, FOLLOWUPS_COL]]
bc_comp_dict = eGFR_mean_IPTW(followups_df, baseline_vars_df, followup_yrs = 5)
'''
weighted std is quite large, and the differences between means are quite small.

To do: 
 - Check the weighted std.
 
'''

In [ ]:
bc_comp_dict.keys()

In [ ]:
bc_comp_dict[('ACEIs', 'ARBs')]

In [ ]:
#compute mean traj curve for "traj_col", but still use the cohort identified by eGFR value

def traj_mean_IPTW(eGFR_followups_df, traj_followups_df, baseline_vars_df, followup_yrs):

    eGFR_followups_df_s, eGFR_baseline_vars_df_s = select_patient_with_followup_yrs(eGFR_followups_df, 
                                                                         baseline_vars_df, 
                                                                         followup_yrs)
    
    #cut data for traj_followups_df
    followup_len = followup_yrs + BASELINE_IDX + 1
    mask = traj_followups_df[FOLLOWUPS_COL].apply(lambda v: True if len(v) >= followup_len else False)
    traj_followups_df_s = traj_followups_df.loc[mask, :].reset_index(drop = True)
    traj_followups_df_s[FOLLOWUPS_COL] = traj_followups_df_s[FOLLOWUPS_COL].apply(lambda v: v[0:followup_len])
    
    eGFR_pids = eGFR_followups_df_s[PID].unique()
    traj_followups_df_s = traj_followups_df_s.loc[traj_followups_df_s[PID].isin(eGFR_pids), :]
    traj_pids = traj_followups_df_s[PID].unique()
    traj_baseline_vars_df_s = eGFR_baseline_vars_df_s.loc[eGFR_baseline_vars_df_s[PID].isin(traj_pids), :]
    
    #check how many patients did not have enough traj length
    print(f'#patients without full traj: {len(eGFR_pids) - len(traj_pids)} ')

    #using weights for eGFR cohorts
    ps_positivity_dict = estimate_propensity_score(eGFR_baseline_vars_df_s, 
                                                   comp_list = ['ACEIs', 'ARBs'])
    
    bc_comp_dict = profile_comparison_IPTW(traj_baseline_vars_df_s, ps_positivity_dict)
    SMD_visualization(bc_comp_dict, followup_yrs)
    
    for pair, ps_df in ps_positivity_dict.items():
        pair_stats_dict = get_pair_stats(traj_followups_df_s, ps_df)
        followups_visualization(pair_stats_dict,
                                ctrl_expolation = False,
                                add_patient_num = False,
                                errorbar = True,
                                v_name = 'Systolic BP')
        
        print(f'-----------------------{pair}------------------------')
        for key, grp_dict in pair_stats_dict.items():
            print(key)
            print('means: ', np.round(grp_dict['means'],2))
            print('CI_lower: ', np.round(grp_dict['CI_lower'],2))
            print('CI_upper: ', np.round(grp_dict['CI_upper'],2),'\n')

    return bc_comp_dict

In [ ]:
followup_yrs = 5
traj_followups_col = TRAJ_COL + '_' + FOLLOWUPS_COL
eGFR_followups_df = cohort_df[[PID, FOLLOWUPS_COL]]
traj_followups_df = cohort_df[[PID, traj_followups_col]].rename(columns = {traj_followups_col: FOLLOWUPS_COL})

In [ ]:
%%time
traj_mean_IPTW(eGFR_followups_df, traj_followups_df, baseline_vars_df, followup_yrs)

In [ ]:
eGFR_followups_df_s, eGFR_baseline_vars_df_s = select_patient_with_followup_yrs(eGFR_followups_df, 
                                                                         baseline_vars_df, 
                                                                         followup_yrs)

In [ ]:
followup_len = followup_yrs + BASELINE_IDX + 1
traj_followups_col = traj_col + '_' + FOLLOWUPS_COL
mask = traj_followups_df[traj_followups_col].apply(lambda v: True if len(v) >= followup_len else False)
traj_followups_df_s = traj_followups_df.loc[mask, :].reset_index(drop = True)
traj_followups_df_s[traj_followups_col] = traj_followups_df_s[traj_followups_col].apply(lambda v: v[0:followup_len])

In [ ]:
eGFR_followups_df_s.shape

In [ ]:
traj_followups_df_s.shape

In [ ]:
eGFR_pids = eGFR_followups_df_s[PID].unique()
traj_pids = traj_followups_df_s[PID].unique()

In [ ]:
len(set(eGFR_pids) - set(traj_pids))

##  KM estimator

### raw 

In [ ]:
# %%time
# KM_estimator_for_params(cohort_df, GRP_LIST)

### IPTW

In [ ]:
def KM_IPTW(cohort_df, baseline_vars_df):

    ps_positivity_dict = estimate_propensity_score(baseline_vars_df, 
                                                   comp_list = ['class_ACEI'])
    
    bc_comp_dict = profile_comparison_IPTW(baseline_vars_df, ps_positivity_dict)
    SMD_visualization(bc_comp_dict, followup_yrs = None)
    
    #extract input data for KM
    for pair, ps_df in ps_positivity_dict.items():
        
        Y_df = cohort_df[[PID, 'event', 'dur']]
        joined_df = Y_df.merge(ps_df, how = 'inner', left_on = PID, right_on = PID)
        
        KM_input_dict = {}
        grps = joined_df[GRP_COL].unique()
        for grp in grps:
            df = joined_df.loc[joined_df[GRP_COL] == grp, :]
            grp_KM_dict = {'events': df['event'].values, 
                           'durations': df['dur'].values,
                           'weights': df['ps_w'].values}
        
            KM_input_dict[grp] = grp_KM_dict
            
        #visualization
        KM_fit_and_visualization(KM_input_dict)

    return bc_comp_dict

In [ ]:
bc_comp_dict = KM_IPTW(cohort_df, baseline_vars_df)

## Cox

### raw

In [ ]:
# %%time
# Cox_for_multiple_events(baseline_vars_df, cohort_df)

### ps-stratification

In [ ]:
# %%time
# ps_positivity_dict = ps_computation(baseline_vars_df)

In [ ]:
# %%time
# Cox_strata_for_multiple_events(ps_positivity_dict, cohort_df, dummies_prefixes)

### ps-matching

In [ ]:
# %%time
# Cox_ps_matching_for_multiple_events(ps_positivity_dict, cohort_df, dummies_prefixes)

###  IPTW

In [ ]:
def Cox_IPTW(cohort_df, baseline_vars_df):
    
    #predefined X_cols
    X_cols = ['gender_Male', 'race_Indian', 'race_Malay',
           'race_OthersX', 'Age', 'DM_diag', 'HLD_diag', 'Macrovascular_diag',
           'Eye_diag', 'Foot_diag', 'Kidney_diag', 'BP_S', 'BP_D', 'HbA1c', 'LDL',
           'BMI', 'DM_biguanides', 'DM_sulfonylureas', 'DM_dpp4_inhibitors',
           'DM_insulin', 'DM_alpha_glucosidase_inhibitors',
           'HLD_hmgcoa_reductase_inhibitors', 'HLD_fibric_acid_derivatives',
           'HLD_cholesterol_absorption_inhibitors', 'HLD_bile_acid_sequestrants']
    
    if eGFR_OPTION == 'stage':
        X_cols = X_cols + ['eGFR_stage_2', 'eGFR_stage_3', 'eGFR_stage_4', 'eGFR_stage_5']
    else:
        X_cols = X_cols + ['eGFR']

    ps_positivity_dict = estimate_propensity_score(baseline_vars_df, 
                                                  comp_list = ['ACEIs','ARBs'])
    
    bc_comp_dict = profile_comparison_IPTW(baseline_vars_df, ps_positivity_dict)
    SMD_visualization(bc_comp_dict, followup_yrs = None)
    
    res_dict = {}
    for pair, ps_df in ps_positivity_dict.items():
        
        #extract and combine data
        X_df = baseline_vars_df[[PID]+X_cols]
        Y_df = cohort_df[[PID, 'dur', 'event']]

        joined_df = join_df_list_on_same_key([X_df, Y_df, ps_df], 
                                             key_list = [PID], 
                                             how='inner')
        joined_df[pair[1]] = (joined_df[GRP_COL] == pair[1]).astype(int)
        joined_df.drop([PID, GRP_COL, 'ps'], axis = 1, inplace = True)
        
        #drop the columns in X_cols if variance = 0\
        X_var = joined_df[X_cols].astype('float64').var(axis = 0)
        var0_cols = list(X_var[X_var == 0].index)
        joined_df.drop(var0_cols, axis = 1, inplace = True)
        
        #fit into the model
        cph = CoxPHFitter(penalizer=0.2)
        cph.fit(joined_df, duration_col = 'dur', event_col = 'event', weights_col = 'ps_w')
        
        summary_df = cph.summary.reset_index()\
                                . rename(columns = {'exp(coef)': 'HR',
                                                   'exp(coef) lower 95%': 'CI_lower', 
                                                   'exp(coef) upper 95%': 'CI_upper', 
                                                   'p': 'P'})\
                                .loc[:,['covariate', 'HR', 'CI_lower', 'CI_upper', 'P']]
        
        res_dict[pair] = summary_df
        
    #extract the HR for compartor
    row_list = []
    for pair, summary_df in res_dict.items():
        comp_row = summary_df.loc[summary_df['covariate'] == pair[1], :].rename(columns = {'covariate': 'comp'})
        comp_row['base'] = pair[0]
        row_list.append(comp_row)
    comp_HR_df = pd.concat(row_list, axis = 0).reset_index(drop = True)
    comp_HR_df = comp_HR_df[['base', 'comp', 'HR', 'CI_lower', 'CI_upper', 'P']]

    return bc_comp_dict, res_dict, comp_HR_df

In [ ]:
baseline_vars_df.columns

In [ ]:
%%time
bc_comp_dict, res_dict, comp_HR_df = Cox_IPTW(cohort_df, baseline_vars_df)

In [ ]:
comp_HR_df

In [ ]:
df_dict_2_excel(bc_comp_dict, file_name = 'baseline_weighitng.xlsx')

In [ ]:
bc_comp_dict[('ACEI', 'ARB')]

In [ ]:
res_dict[('Others','class_ARB')].round(2)

# Sub-analysis by conditions

**Input**:
- `baseline_vars_df`: for data selection, and ps estimation
- `cohort_df`: to obtain the survival labels
- `conds`: to specify a set of selection condition. The default logical relationship between two conditions is AND. It is a list of cond_tuples. cond_tuple = (col, op, value)
    - col: str
    - op: str, e.g., '==', '<='
    - value: bool/str/int/float/datetime

**Output:**
- For each (base, comp) pair, a forest plot is generated with visualized HR, 95% CI and P.

In [ ]:
def selection_by_conds(df, conds):
    
    import operator
    ops = {
        '==': operator.eq,
        '>': operator.gt,
        '>=': operator.ge,
        '<': operator.lt,
        '<=': operator.le
    }
    
    n = len(df)
    mask = np.array([True] * n)
    
    for col, op, value in conds:
        func = ops[op]
        mask = mask & func(df[col], value)
        
    return df.loc[mask, :]

In [ ]:
def float2str_list(nums):
    
    return ["{:.2f}".format(i) for i in nums]

In [ ]:
def forest_plot(df):
    
    import zepid
    from zepid.graphics import EffectMeasurePlot
    
    base, comp = df.name
    labels = df['cond'].values
    HRs = float2str_list(df['HR'].values)
    CI_lower = float2str_list(df['CI_lower'].values)
    CI_upper = float2str_list(df['CI_upper'].values)

    #mark the statistical significant on the label
    P_values = df['P'].values
    for i in range(0,len(labels)):
        if P_values[i] < 0.05:
            labels[i] += '*'
        else:
            labels[i] += ' '
    
    p = EffectMeasurePlot(label=labels, effect_measure=HRs, lcl=CI_lower, ucl=CI_upper)
    p.labels(effectmeasure='HR')
    p.labels(scale='log')
    #t_adjuster: alignment of the tables, should be adjusted manually
    ax=p.plot(figsize=(7,4), t_adjuster=0.02, max_value=1.5, min_value=0.7)
    ax.set_xlabel(f'Favours {comp}              Favours {base} ', fontsize=10)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['bottom'].set_visible(True)
    ax.spines['left'].set_visible(False)
#     plt.tight_layout()
    plt.show()
    
    return

In [ ]:
def subanalysis_by_conds(cohort_df, baseline_vars_df, conds_dict):
    
    HR_df_list = []
    for cond_name, conds in conds_dict.items():
        
        baseline_vars_df_s = selection_by_conds(baseline_vars_df, conds)
        bc_comp_dict, res_dict, pair_HR_df = Cox_IPTW(cohort_df, baseline_vars_df_s)
        pair_HR_df = pair_HR_df.round(2)
        pair_HR_df['cond'] = cond_name
        HR_df_list.append(pair_HR_df)
        
    comb_res = pd.concat(HR_df_list, axis = 0).reset_index(drop = True)
    
    comb_res.groupby(['base', 'comp']).apply(lambda df: forest_plot(df))

    return comb_res

In [ ]:
if eGFR_OPTION == 'stage':
    conds_dict = {
        'Whole cohort': [],
        'Male': [('gender_Male', '==', True)],
        'Female': [('gender_Female', '==', True)],
        'Age < 70': [('Age', '<', 70)],
        'Age >= 70': [('Age', '>=', 70)],
        'Systolic BP < 140': [('BP_S', '<', 140)],
        'Systolic BP >= 140':  [('BP_S', '>=', 140)],
        'CKD stage 1': [('eGFR_stage_1', '==', True)],
        'CKD stage 2': [('eGFR_stage_2', '==', True)],
        'CKD stage 3-5': [('eGFR_stage_1', '==', False), ('eGFR_stage_2', '==', False)],
        'Diabetes Yes': [('DM_diag', '==', True)],
        'Diabetes No': [('DM_diag', '==', False)],
        'Kidney disease Yes': [('Kidney_diag', '==', True)],
        'Kidney disease No': [('Kidney_diag', '==', False)],
        'Marcovascular disease Yes': [('Macrovascular_diag', '==', True)],
        'Marcovascular disease No': [('Macrovascular_diag', '==', False)]
    }
else:
    conds_dict = {
        'Whole cohort': [],
        'Male': [('gender_Male', '==', True)],
        'Female': [('gender_Female', '==', True)],
        'Age < 70': [('Age', '<', 70)],
        'Age >= 70': [('Age', '>=', 70)],
        'Systolic BP < 140': [('BP_S', '<', 140)],
        'Systolic BP >= 140':  [('BP_S', '>=', 140)],
        'CKD stage 1': [('eGFR', '>=', 90)],
        'CKD stage 2': [('eGFR', '>=', 60), ('eGFR', '<', 90)],
        'CKD stage 3-5': [('eGFR', '<', 60)],
        'Diabetes Yes': [('DM_diag', '==', True)],
        'Diabetes No': [('DM_diag', '==', False)],
        'Kidney disease Yes': [('Kidney_diag', '==', True)],
        'Kidney disease No': [('Kidney_diag', '==', False)],
        'Marcovascular disease Yes': [('Macrovascular_diag', '==', True)],
        'Marcovascular disease No': [('Macrovascular_diag', '==', False)]
    }

comb_res = subanalysis_by_conds(cohort_df, baseline_vars_df, conds_dict)

In [ ]:
comb_res

In [ ]:
comb_res.groupby(['base', 'comp']).apply(lambda df: forest_plot(df))